In [1]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
import torch

In [13]:
test_data = pd.read_csv("result_data/tft_data_36_test.csv").tail(36)

In [14]:
test_data

,Unnamed: 0,ss_id,init_time,step,pv_datetime,pv_hour,horizon,generation,capacity,normalize_generation,...,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100
0,48348,26985.0,2018-11-15 12:00:00,0 days 00:00:00,2018-11-15 16:00:00,16.0,1.0,0.000000,4.0,0.000000,...,0.963867,0.261536,0.0,0.000,286.73633,0.971954,-2.419031,-3.132269,5.898019,8.501874
1,48349,26985.0,2018-11-15 12:00:00,0 days 01:00:00,2018-11-15 17:00:00,17.0,2.0,0.000000,4.0,0.000000,...,1.000000,0.115631,0.0,526976.000,287.06274,1.000000,-2.717444,-3.569618,5.633824,7.929080
2,48350,26985.0,2018-11-15 12:00:00,0 days 02:00:00,2018-11-15 18:00:00,18.0,3.0,0.000000,4.0,0.000000,...,1.000000,0.020874,0.0,133376.000,286.91820,1.000000,-2.529895,-3.632496,5.222868,7.941486
3,48351,26985.0,2018-11-15 12:00:00,0 days 03:00:00,2018-11-15 19:00:00,19.0,4.0,0.000000,4.0,0.000000,...,0.998901,0.215607,0.0,11136.000,286.92578,0.999115,-2.069286,-2.887247,4.480828,7.139501
4,48352,26985.0,2018-11-15 12:00:00,0 days 04:00:00,2018-11-15 20:00:00,20.0,5.0,0.000000,4.0,0.000000,...,1.000000,0.305634,0.0,35328.000,286.54468,1.000000,-2.679039,-3.784738,4.528837,7.023499
5,48353,26985.0,2018-11-15 12:00:00,0 days 05:00:00,2018-11-15 21:00:00,21.0,6.0,0.000000,4.0,0.000000,...,1.000000,0.000397,0.0,1536.000,285.87524,1.000000,-1.826510,-2.433717,4.379947,7.079319
6,48354,26985.0,2018-11-15 12:00:00,0 days 06:00:00,2018-11-15 22:00:00,22.0,7.0,0.000000,4.0,0.000000,...,1.000000,0.006256,0.0,0.000,285.74487,1.000000,-1.783433,-2.027638,3.811839,6.439410
7,48355,26985.0,2018-11-15 12:00:00,0 days 07:00:00,2018-11-15 23:00:00,23.0,8.0,0.000000,4.0,0.000000,...,1.000000,0.000000,0.0,0.000,285.65234,1.000000,-1.745071,-1.883446,3.011540,5.385149
8,48356,26985.0,2018-11-15 12:00:00,0 days 08:00:00,2018-11-16 00:00:00,0.0,9.0,0.000000,4.0,0.000000,...,1.000000,0.000000,0.0,0.000,285.51490,1.000000,-1.773601,-2.055286,2.994912,5.313138
9,48357,26985.0,2018-11-15 12:00:00,0 days 09:00:00,2018-11-16 01:00:00,1.0,10.0,0.000000,4.0,0.000000,...,1.000000,0.000000,0.0,0.000,285.42310,1.000000,-1.546715,-1.707959,3.060022,5.426452


In [15]:
test_data = test_data.drop(columns=['Unnamed: 0'])
test_data.columns

Index(['ss_id', 'init_time', 'step', 'pv_datetime', 'pv_hour', 'horizon',
       'generation', 'capacity', 'normalize_generation', 'lat', 'long', 'tilt',
       'orientation', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde',
       'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100'],
      dtype='object')

In [16]:
best_model_path = 'lightning_logs/my_model/version_20/checkpoints/epoch=49-step=1500.ckpt'
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)


In [17]:
max_encoder_length = 36
max_prediction_length = 36

test_data.rename(columns={'pv_hour': 'day_hour'}, inplace=True)

test_data['ss_id'] = test_data['ss_id'].astype(int)

test_data['pv_datetime'] = pd.to_datetime(test_data['pv_datetime'])
test_data['date'] = test_data['pv_datetime'].dt.date
test_data['day_of_week'] = test_data['pv_datetime'].dt.dayofweek
test_data['month'] = test_data['pv_datetime'].dt.month

test_data['ss_id'] = test_data['ss_id'].astype(str)
test_data['day_of_week'] = test_data['day_of_week'].astype(str)
test_data['month'] = test_data['month'].astype(str)
test_data['day_hour'] = test_data['day_hour'].astype(str)

test_data['time_idx'] = test_data.index
test_data['time_idx'] = test_data['time_idx'].astype(int)

In [18]:
test_data.shape

(36, 31)

In [19]:
new_data = TimeSeriesDataSet(
    test_data,
    time_idx="time_idx",
    target="normalize_generation",
    group_ids=["ss_id"],  # Grouping by ss_id to identify different PV sites
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_reals=["capacity", "lat", "long", "tilt", "orientation"],
    time_varying_known_categoricals=[ "month", "day_of_week", "day_hour"],
    time_varying_known_reals=["time_idx", "dlwrf", "dswrf", "duvrs", "hcc", "lcc", "mcc", "sde", "sr", "t2m", "tcc", "u10", "u100", "v10", "v100"],
    time_varying_unknown_reals=["normalize_generation"],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=False,
)


In [20]:
new_data_loader = new_data.to_dataloader(train=False, batch_size=128, num_workers=2)

In [21]:
# lightning_logs/my_model/version_20/checkpoints/epoch=49-step=1500.ckpt
# actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
# predictions = best_tft.predict(val_dataloader)

In [22]:
actuals = torch.cat([y[0] for x, y in iter(new_data_loader)])

In [23]:
predictions = best_tft.predict(new_data_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [24]:
print("Checking for NaN values in actuals and predictions...")
print("Number of NaNs in actuals:", torch.isnan(actuals).sum().item())
print("Number of NaNs in predictions:", torch.isnan(predictions).sum().item())

Checking for NaN values in actuals and predictions...
Number of NaNs in actuals: 0
Number of NaNs in predictions: 306


In [25]:

mae = (actuals - predictions).abs().mean().item()
print(f"Mean Absolute Error: {mae}")


average_p50_loss = (actuals - predictions).abs().mean(axis=1)
print(f"Average P50 Loss per Time Series: {average_p50_loss}")


Mean Absolute Error: nan
Average P50 Loss per Time Series: tensor([18.0284,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan])


In [26]:
print("Actuals:", actuals.tolist())
print("Predictions:", predictions.tolist())

Actuals: [[66.15959930419922, 63.96870040893555, 77.85449981689453, 53.927101135253906, 31.880699157714844, 6.67779016494751, 0.000695820024702698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [63.96870040893555, 77.85449981689453, 53.927101135253906, 31.880699157714844, 6.67779016494751, 0.000695820024702698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [77.85449981689453, 53.927101135253906, 31.880699157714844, 6.67779016494751, 0.000695820024702698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [53.927101135253906, 31.880699157714844, 6.67779016494751, 0.000695820024702698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [31.880699157714844, 6.67779016494751, 0.000695820024702698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [6.67779016494751, 0.000695820024702698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.000695820024702698, 0.0, 0.0, 0.0, 0

In [30]:
print(actuals)

tensor([[6.6160e+01, 6.3969e+01, 7.7854e+01, 5.3927e+01, 3.1881e+01, 6.6778e+00,
         6.9582e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [6.3969e+01, 7.7854e+01, 5.3927e+01, 3.1881e+01, 6.6778e+00, 6.9582e-04,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [7.7854e+01, 5.3927e+01, 3.1881e+01, 6.6778e+00, 6.9582e-04, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [5.3927e+01, 3.1881e+01, 6.6778e+00, 6.9582e-04, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.1881e+01, 6.6

In [27]:
mask = ~torch.isnan(actuals) & ~torch.isnan(predictions)
actuals_filtered = actuals[mask]
predictions_filtered = predictions[mask]

# Calculate MAE
mae = (actuals_filtered - predictions_filtered).abs().mean().item()
print(f"Mean Absolute Error: {mae}")


Mean Absolute Error: 18.83304786682129


In [28]:
print("Shape of actuals:", actuals.shape)
print("Shape of predictions:", predictions.shape)


Shape of actuals: torch.Size([35, 18])
Shape of predictions: torch.Size([35, 18])


In [29]:
print(f"Actuals size: {actuals.size()}")
print(f"Predictions size: {predictions.size()}")

Actuals size: torch.Size([35, 18])
Predictions size: torch.Size([35, 18])
